In [1]:
%load_ext autoreload
%autoreload 2

IMPORTS
==================



In [2]:
import pandas as pd
import os
import pickle
from pyprojroot import here

In [3]:
from src.tauso.off_target.Roni.off_target_pipeline.off_target_functions import parse_gtf, parse_fasta

from src.tauso.off_target.Roni.off_target_pipeline.get_premRNA_sequences import general_exp_data, enrich_expression_data_with_sequence

from src.tauso.off_target.Roni.off_target_pipeline.mutate_cell_line_transcriptome import get_expression_of_cell_line, get_mutations_of_cell_line, mutate, mutation_dict, celline_list, find_shift, mutate_transcriptome

DATA
====

Paths

In [5]:
PROJECT_ROOT = here()
DATA_DIR = os.path.join(PROJECT_ROOT, "src", "tauso", "off_target", "Roni", "data")
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "src", "tauso", "off_target", "Roni", "outputs")

ASO_DATA_DIR = os.path.join(PROJECT_ROOT, "notebooks", "data")
aso_data_path = os.path.join(ASO_DATA_DIR, "data_asoptimizer_updated.csv")
aso_df = pd.read_csv(aso_data_path)

fasta_path = os.path.join(DATA_DIR, "GRCh38.p13.genome.fa")
gtf_path = os.path.join(DATA_DIR, "gencode.v34.chr_patch_hapl_scaff.annotation.gtf")
exp_path = os.path.join(DATA_DIR, "OmicsExpressionTPMLogp1HumanProteinCodingGenes.csv")
mut_path = os.path.join(DATA_DIR, "OmicsSomaticMutations.csv")

/tmp/ipykernel_5459/2562656009.py:7: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  aso_df = pd.read_csv(aso_data_path)


Select the cell-line(s) you work on

In [16]:
save_csv = True

In [17]:
#cell_line_lst = ["ACH-000681"]
cell_line_lst = ["general", "ACH-001328", "ACH-000463", "ACH-001188", "ACH-001086", "ACH-000739", "ACH-000232"]
transcriptomes = {}

# GENERAL TRANSCRIPTOME

I. GET CELL-LINE TRANSCRIPTOME SEQUENCES
=

In [18]:
# Step 1 - Parse FASTA
chrom2seq = parse_fasta(fasta_path, "../outputs/fasta_pickle_output_file_path.pkl")

# Step 2 - Parse GTF
annotations = parse_gtf(gtf_path, "../outputs/gtf_pickle_output_file_path.pkl")

KeyboardInterrupt: 

In [ ]:
# Step 3 - Expression and mutation data per cell line
for cell_line in cell_line_lst:

    exp_data = get_expression_of_cell_line(cell_line, exp_path, OUTPUT_DIR)
    mut_data = get_mutations_of_cell_line(cell_line, mut_path, OUTPUT_DIR)

    transcriptomes[cell_line] = [exp_data, mut_data]
    transcriptomes = enrich_expression_data_with_sequence(transcriptomes, cell_line, chrom2seq, annotations)

    exp_df = transcriptomes[cell_line][0]
    exp_df["Original Transcript Sequence"] = (exp_df["Original Transcript Sequence"].astype(str).str.replace("T", "U"))
    mut_df = transcriptomes[cell_line][1]

    mut_exp_data = mutate_transcriptome(exp_df, mut_df, annotations)
    transcriptomes[cell_line].append(mut_exp_data)

    if save_csv:
        for cell_line, data in transcriptomes.items():
            df = pd.DataFrame(data[-1])
            df.to_csv(os.path.join(OUTPUT_DIR, cell_line + "_transcriptome.csv"), index=False)


Successfully processed ACH-001328. Found 15270 expressed genes.
Obtained mutation data for ACH-001328
Building Gene -> Transcript index...
Processing ACH-001328...
Enrichment complete.
Mutating transcriptomes...
ENST00000379116.10 shift: 8766
really mutated ENST00000379116.10
ENST00000378536.5 shift: 448
really mutated ENST00000378536.5
ENST00000378378.9 shift: 8420
really mutated ENST00000378378.9
ENST00000333172.11 shift: 18826
really mutated ENST00000333172.11
ENST00000333172.11 shift: 18826
didnt really do anything... check ENST00000333172.11
ENST00000376259.7 shift: 25321
really mutated ENST00000376259.7
ENST00000376259.7 shift: 25321
didnt really do anything... check ENST00000376259.7
ENST00000375254.8 shift: 107078
really mutated ENST00000375254.8
ENST00000375254.8 shift: 107078
didnt really do anything... check ENST00000375254.8
ENST00000373912.8 shift: 7176
really mutated ENST00000373912.8
ENST00000373912.8 shift: 7176
didnt really do anything... check ENST00000373912.8
ENST00

In [9]:
exp_data = general_exp_data(exp_path)
mut_data = pd.DataFrame
transcriptomes["general"] = [exp_data, mut_data]

transcriptomes = enrich_expression_data_with_sequence(transcriptomes, "general", chrom2seq, annotations)

exp_df = transcriptomes["general"][0]
exp_df["Original Transcript Sequence"] = (
exp_df["Original Transcript Sequence"].astype(str).str.replace("T", "U"))
exp_df["Mutated Transcript Sequence"] = None

if save_csv:
    df = pd.DataFrame(transcriptomes["general"][0])
    df.to_csv(os.path.join(OUTPUT_DIR, "general" + "_transcriptome.csv"), index=False)


Building Gene -> Transcript index...
Processing general...
Enrichment complete.


In [11]:
if save_csv:
    df = pd.DataFrame(transcriptomes["general"][0])
    df.to_csv(os.path.join(OUTPUT_DIR, "general" + "_transcriptome.csv"), index=False)

III. OFF-TARGET CALCULATION
=

In [6]:

from src.tauso.off_target.Roni.off_target_pipeline.off_target_feature import run_off_target_pipeline

add off target feat


In [7]:
cell_line_list = ['ACH-001328', 'ACH-000463', 'ACH-001188', 'ACH-001086', 'ACH-000739', 'ACH-000232']

cellline_name_to_depmap = {
    'A431': 'ACH-001328',
    'A-431': 'ACH-001328',
    'NCI-H460': 'ACH-000463',
    'SH_SY5Y': 'ACH-001188',
    'SH-SY5Y': 'ACH-001188',
    'HeLa': 'ACH-001086',
    'Hela': 'ACH-001086',
    'HepG2': 'ACH-000739',
    'U-251MG': 'ACH-000232',
    'U251': 'ACH-000232'
}



CREATE TRANSCRIPTOMES DIRECTLY


In [9]:

transcriptomes = {}

transcriptomes['general'] = [pd.read_csv(os.path.join(OUTPUT_DIR, "general_transcriptome.csv"))]
for cell_line in cell_line_list:
    exp_data = pd.read_csv(os.path.join(OUTPUT_DIR, cell_line + "_expression.csv"))
    mut_data = pd.read_csv(os.path.join(OUTPUT_DIR, cell_line + "_mutations.csv"))
    transcriptome = pd.read_csv(os.path.join(OUTPUT_DIR, cell_line + "_transcriptome.csv"))
    transcriptomes[cell_line] = [exp_data, mut_data, transcriptome]


In [10]:
transcriptomes.keys()

dict_keys(['general', 'ACH-001328', 'ACH-000463', 'ACH-001188', 'ACH-001086', 'ACH-000739', 'ACH-000232'])

In [11]:
run_off_target_pipeline(
    ASO_df=aso_df,
    cell_line2data=transcriptomes ,
    output_dir=OUTPUT_DIR, top_n_list=[75, 100, 125],
    cutoff_list=[1100, 1200, 1300], method=0,
    name_to_depmap=cellline_name_to_depmap
)
# run_off_target_pipeline(
#     ASO_df=aso_df,
#     cell_line2data=transcriptomes ,
#     output_dir=OUTPUT_DIR, top_n_list=[10],
#     cutoff_list=[200], method=0,
#     name_to_depmap=cellline_name_to_depmap
# )

Running config: TopN=75, Cutoff=1100
Phase 1: Computing General Scores and Caching Energies...
Phase 2: Computing Specific Scores using Baseline + Delta...
Running config: TopN=75, Cutoff=1200
Phase 1: Computing General Scores and Caching Energies...
Phase 2: Computing Specific Scores using Baseline + Delta...
Running config: TopN=75, Cutoff=1300
Phase 1: Computing General Scores and Caching Energies...
Phase 2: Computing Specific Scores using Baseline + Delta...
Running config: TopN=100, Cutoff=1100
Phase 1: Computing General Scores and Caching Energies...
Phase 2: Computing Specific Scores using Baseline + Delta...
Running config: TopN=100, Cutoff=1200
Phase 1: Computing General Scores and Caching Energies...
Phase 2: Computing Specific Scores using Baseline + Delta...
Running config: TopN=100, Cutoff=1300
Phase 1: Computing General Scores and Caching Energies...
Phase 2: Computing Specific Scores using Baseline + Delta...
Running config: TopN=125, Cutoff=1100
Phase 1: Computing Gene